<H1> CAFE24 Crawling with Selenium  </H1>

1.Anaconda Selenium error 
ModuleNotFoundError: No module named 'selenium'
  
2.Failed - installing selenium (1)
파이참의 동작 코드와 위와 같은 문제를 일으켜서, 
pip3 install selenium 를 수행했으나, 여전히 같았다. 

3.Referneces
https://devminjun.github.io/blog/Selenium

4.download chrome driver 
https://chromedriver.storage.googleapis.com/index.html?path=2.40/

5.OK- installing selenium
 ~/anaconda3/bin/pip install selenium
 
6.add permission to chromedriver
permission 문제가 생겨서 chromedriver에 777을 부여 

7.chrome을 찾지 못함
WebDriverException: Message: unknown error: cannot find Chrome binary
  (Driver info: chromedriver=2.40.565383 (76257d1ab79276b2d53ee976b2c3e3b9f335cde7),platform=Linux 4.4.0-127-generic x86_64)
  
8.installing chronium-browser
sudo apt install chromium-browser
이렇게 하니, 진해이 된다. 이것저것 많이도 깐다. 

9.using headless crawling
headless하게 하는 방법에 대해서 설명하고 있다. 
https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/

--headless 를 붙이니까, 일단은 성공하는 듯하게 보인다. 

In [1]:
from selenium import webdriver
import time
from time import sleep
from bs4 import BeautifulSoup

import sys 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import folium
import json
import warnings
warnings.simplefilter(action='ignore' ,  category=FutureWarning)

#references : http://selenium-python.readthedocs.io/

#path_chrome_driver = "D:\\Workspace\\temp\\DataScience-GitData\\DataScience-master\\chromedriver"
path_chrome_driver = "/root/chromedriver/chromedriver"


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--window-size=1920x1080')
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox") 

In [ ]:
#for resource
import platform
from matplotlib import font_manager,rc
def checkFontResource()  :
    path = "c:\\Windows\\Fonts\\malgun.ttf"

    if platform.system() == "Darwin" :
        rc('font', family='AppleGothic')
    elif platform.system() == "Windows" :
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family = font_name)
    else :
        print("unknown system... sorry~~~~")
    plt.rcParams['axes.unicode_minus'] = False

def crawling_sendkey( driver, by , name, sendvalue ) :
    if( by.lower() == "name ") : elem = driver.find_element_by_name(name)
    elif (by.lower() == "xpath"): elem = driver.find_element_by_xpath(name)
    else : return False
    elem.clear()
    elem.send_keys(sendvalue)
    sleep(0.1)
    return True;

def crawling_select_getlist( driver, xpath ) :
    list = driver.find_element_by_xpath(xpath).find_elements_by_tag_name("option")
    value = [option.text for option in list]
    value = value[1:]  # 전체 제거?
    sleep(0.5)  # this makes correct result
    return value

def crawling_table_getlist_tag_xpath(driver, xpath, tag , isElemList ):
    list_webelem = driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag)
    if isElemList == True : return list_webelem
    else:  return [ {"text":elem.text, "href":elem.get_attribute("href"), "elem":elem , "src":elem.get_attribute("src")} for elem in list_webelem ]

def crawling_select_Item( driver, xpath , selectId , order ) :
    list = driver.find_element_by_xpath(xpath).find_elements_by_tag_name("option")
    value = [option.text for option in list]
    value = value[1:]  # 전체 제거?
    element = driver.find_element_by_id( selectId )
    element.send_keys(value[order])
    sleep(0.5)  # this makes correct result

def crawling_click( driver , xpath  ) :
    driver.find_element_by_xpath(xpath).click()
    sleep(0.5)

def InitWebDriver( WebDriverPath, URL , delay ) :
    global webDriver
    webDriver = webdriver.Chrome(WebDriverPath ,chrome_options=options)
    webDriver.get(URL)
    webDriver.implicitly_wait(delay)
    return True

def Main() :
    global path_chrome_driver
    checkFontResource()
    #go to url with initalized webdriver
    
    print("1")
    
    InitWebDriver( path_chrome_driver , "http://www.bookcosmos.com" , 1 )
    #login
    crawling_sendkey(webDriver, "xpath", "/html/body/div/div[2]/div/ul[1]/li[1]/form/div/div[1]/input[1]", "hamtorigun" )
    crawling_sendkey(webDriver, "xpath", "/html/body/div/div[2]/div/ul[1]/li[1]/form/div/div[1]/input[2]", "saturn")
    crawling_click(webDriver,"//*[@id=\"btnLogin\"]" )
    crawling_click(webDriver, "/html/body/div/div[2]/div/ul[1]/li[2]/a")

    sleep(0.5)

    listA = []
    listB = []

    print("2")
    
    # table 1 에서 href와 Text 얻어내기
    #print( "len of table1's href ", len (webDriver.find_element_by_xpath("/html/body/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody/tr/td/table[4]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[2]/tbody/tr[1]/td/table[1]/tbody").find_elements_by_tag_name("a") ) )

    listA = crawling_table_getlist_tag_xpath( webDriver , "/html/body/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody/tr/td/table[4]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[2]/tbody/tr[1]/td/table[1]" , "a" , False )
    listB = crawling_table_getlist_tag_xpath( webDriver , "/html/body/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody/tr/td/table[4]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[2]/tbody/tr[1]/td/table[2]" , "a" , False )
    #listA[1]["elem"].click()

    xpath_lastPage  = "/html/body/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody/tr/td/table[4]/tbody/tr/td[2]/table[2]/tbody/tr/td[2]/table/tbody/tr[2]/td/a[11]/img"
    xpath_lastPage2 = "/html/body/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody/tr/td/table[4]/tbody/tr/td[2]/table[2]/tbody/tr/td[2]/table/tbody/tr[2]/td/a[10]/img"

    xpath_scrollBar = "/html/body/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody/tr/td/table[4]/tbody/tr/td[2]/table[2]/tbody/tr/td[2]/table/tbody/tr[2]/td"

    if False :
        for elem in listA :
            print( elem )
            webDriver.get(elem["href"])
            sleep(0.5)
            crawling_click(webDriver, xpath_lastPage2)
            sleep(0.5)
        for elem in listB :
            print(elem)
            webDriver.get(elem["href"])
            sleep(0.5)
            crawling_click(webDriver, xpath_lastPage2)
            sleep(0.5)
    else :        
        print("3")        
        webDriver.get(listA[2]["href"])
        if False :
            crawling_click( webDriver, xpath_lastPage )
            aList = crawling_table_getlist_tag_xpath(webDriver, xpath_scrollBar , "a", False)
            strongList = crawling_table_getlist_tag_xpath(webDriver, xpath_scrollBar , "strong", False)
        else :
            imgList = crawling_table_getlist_tag_xpath(webDriver, xpath_scrollBar, "img", False)
            aList = crawling_table_getlist_tag_xpath(webDriver, xpath_scrollBar, "a", False)
            strongList = crawling_table_getlist_tag_xpath(webDriver, xpath_scrollBar, "strong", False)
            print( imgList )
            print( aList )
            print( strongList ) # current cursor
    exit(0)
    
Main()